In [1]:
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import glob
import torch
import shutil
import sys
import random
from joblib import Parallel, delayed
from IPython.display import display, HTML
from matplotlib import animation, rc
from pathlib import Path

sys.path.append('../input/tensorflow-great-barrier-reef')
rc('animation', html='jshtml')

In [2]:
!pip install -qU bbox-utility

In [3]:
# check https://github.com/awsaf49/bbox for source code of following utility functions
from bbox.utils import coco2yolo, coco2voc, voc2yolo
from bbox.utils import draw_bboxes, load_image
from bbox.utils import clip_bbox, str2annot, annot2str

# 1. sequence split

In [36]:
FOLD      = 1
DIM       = 1280
RANDOM_SEED  = 42
NOBBOX = 0.5
SOURCE_DIR  = '../input/tensorflow-great-barrier-reef/'
DATA_DIR = f'../input/yolov5_public_fold{FOLD}_nobbox{int(NOBBOX*100):}%/'

In [37]:
(Path(DATA_DIR) / "images/train").mkdir(parents=True, exist_ok=True)
(Path(DATA_DIR) / "images/val").mkdir(parents=True, exist_ok=True)
(Path(DATA_DIR) / "labels/train").mkdir(parents=True, exist_ok=True)
(Path(DATA_DIR) / "labels/val").mkdir(parents=True, exist_ok=True)

In [38]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(RANDOM_SEED)

In [39]:
# Train Data
df = pd.read_csv(f'{SOURCE_DIR}train.csv')
df['old_image_path'] = f'{SOURCE_DIR}train_images/video_'+df.video_id.astype(str)+'/'+df.video_frame.astype(str)+'.jpg'
df['annotations'] = df['annotations'].apply(eval)
df['num_bbox'] = df['annotations'].progress_apply(lambda x: len(x))
data = (df.num_bbox>0).value_counts(normalize=True)*100
print(f"No BBox: {data[0]:0.2f}% | With BBox: {data[1]:0.2f}%")

  0%|          | 0/23501 [00:00<?, ?it/s]

No BBox: 79.07% | With BBox: 20.93%


In [40]:
# Drop nobbox images
df['num_annotations'] = df['annotations'].apply(len)


sub1 = df.loc[df.num_annotations > 0].copy()
sub2 = df.loc[df.num_annotations == 0].copy()

sub2 = sub2.sample(frac=NOBBOX, random_state=RANDOM_SEED, replace=False)
df = pd.concat([sub1, sub2], axis=0)

df = df.reset_index(drop=True)

In [41]:
def get_bbox(annots):
    bboxes = [list(annot.values()) for annot in annots]
    return bboxes

def get_imgsize(row):
    row['width'], row['height'] = imagesize.get(row['image_path'])
    return row

colors = [(np.random.randint(255), np.random.randint(255), np.random.randint(255))\
          for idx in range(1)]

In [42]:
df['bboxes'] = df.annotations.progress_apply(get_bbox)
df['width']  = 1280
df['height'] = 720

  0%|          | 0/14210 [00:00<?, ?it/s]

In [43]:
folds = pd.read_csv("../input/train-5folds.csv")
df = df.merge(folds[['image_id', 'fold']], how='left')

In [44]:
df['fold'] = df['video_id'].apply(lambda x: 'train' if x != FOLD else 'val')

In [45]:
df['image_path'] = f'{DATA_DIR}'+ 'images/'+ df.fold + '/' + df.image_id+'.jpg'
df['label_path'] = f'{DATA_DIR}'+ 'labels/'+ df.fold + '/' + df.image_id+'.txt'

In [46]:
df.to_csv(DATA_DIR + "train.csv")

In [47]:
cnt = 0
all_bboxes = []
bboxes_info = []
for row_idx in tqdm(range(df.shape[0])):
    row = df.iloc[row_idx]
    image_height = row.height
    image_width  = row.width
    bboxes_coco  = np.array(row.bboxes).astype(np.float32).copy()
    num_bbox     = len(bboxes_coco)
    names        = ['cots']*num_bbox
    labels       = np.array([0]*num_bbox)[..., None].astype(str)
    ## Create Annotation(YOLO)
    with open(row.label_path, 'w') as f:
        if num_bbox<1:
            annot = ''
            f.write(annot)
            cnt+=1
            continue
        bboxes_voc  = coco2voc(bboxes_coco, image_height, image_width)
        bboxes_voc  = clip_bbox(bboxes_voc, image_height, image_width)
        bboxes_yolo = voc2yolo(bboxes_voc, image_height, image_width).astype(str)
        all_bboxes.extend(bboxes_yolo.astype(float))
        bboxes_info.extend([[row.image_id, row.video_id, row.sequence]]*len(bboxes_yolo))
        annots = np.concatenate([labels, bboxes_yolo], axis=1)
        string = annot2str(annots)
        f.write(string)
print('Missing:',cnt)

def make_copy(row):
    shutil.copyfile(row.old_image_path, row.image_path)
    return


image_paths = df.old_image_path.tolist()
_ = Parallel(n_jobs=-1, backend='threading')(delayed(make_copy)(row) for _, row in tqdm(df.iterrows(), total=len(df)))

  0%|          | 0/14210 [00:00<?, ?it/s]

Missing: 9291


  0%|          | 0/14210 [00:00<?, ?it/s]

In [48]:
%%writefile ../input/yolov5_public_fold{FOLD}_nobbox{int(NOBBOX*100):}%/data.yaml

path: ../input/yolov5_public_fold1  # dataset root dir
train: images/train  # train images (relative to 'path') 128 images
val: images/val  # val images (relative to 'path') 128 images
test:  # test images (optional)

# Classes
nc: 1  # number of classes
names: ['cots']  # class names

Writing ../input/yolov5_public_fold1_nobbox50%/data.yaml


# 2. video split

In [72]:
VIDEO    = 0
SOURCE_DIR  = '../input/tensorflow-great-barrier-reef/'
DATA_DIR = f'../input/yolov5_video{VIDEO}/'

In [73]:
train = pd.read_csv(SOURCE_DIR + 'train.csv')
train['pos'] = train.annotations != '[]'

In [74]:
train['fold'] = train['video_id'].apply(lambda x: "val" if x == VIDEO else "train")

In [75]:
train.head()

,video_id,sequence,video_frame,sequence_frame,image_id,annotations,pos,fold
0,0,40258,0,0,0-0,[],False,val
1,0,40258,1,1,0-1,[],False,val
2,0,40258,2,2,0-2,[],False,val
3,0,40258,3,3,0-3,[],False,val
4,0,40258,4,4,0-4,[],False,val


In [77]:
!mkdir -p {DATA_DIR}/images/val
!mkdir -p {DATA_DIR}/images/train

!mkdir -p {DATA_DIR}/labels/val
!mkdir -p {DATA_DIR}/labels/train

In [78]:
train.to_csv(DATA_DIR+"train.csv", index=None)

In [ ]:
from shutil import copyfile
fold = VIDEO

annos = []
for i, x in train.iterrows():
    if x.video_id == fold:
        mode = 'val'
    else:
        # train
        mode = 'train'
        if not x.pos: continue
        # val
    copyfile(f'../input/tensorflow-great-barrier-reef/train_images/video_{x.video_id}/{x.video_frame}.jpg',
                f'{DATA_DIR}/images/{mode}/{x.image_id}.jpg')
    if not x.pos:
        continue
    r = ''
    anno = eval(x.annotations)
    for an in anno:
#            annos.append(an)
        r += '0 {} {} {} {}\n'.format((an['x'] + an['width'] / 2) / 1280,
                                        (an['y'] + an['height'] / 2) / 720,
                                        an['width'] / 1280, an['height'] / 720)
    with open(f'{DATA_DIR}/labels/{mode}/{x.image_id}.txt', 'w') as fp:
        fp.write(r)

In [71]:
%%writefile {DATA_DIR}/data.yaml

path: '../input/yolov5_video0/'  # dataset root dir
train: images/train  # train images (relative to 'path') 128 images
val: images/val  # val images (relative to 'path') 128 images
test:  # test images (optional)

# Classes
nc: 1  # number of classes
names: ['cots']  # class names

Overwriting ../input/yolov5_video1//data.yaml
